In [ ]:
import os
import numpy as np
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import reproject

# ===== USER SETTINGS =====
SCENE_DIR = "SR_outputs"
OUT_DIR = os.path.join(SCENE_DIR, "pansharpened_Brovey_basic")
os.makedirs(OUT_DIR, exist_ok=True)

MS_BANDS = [1, 2, 3, 4, 5, 7]  # Multispectral bands
PAN_BAND = 8                   # Panchromatic band

# ===== HELPER FUNCTION =====
def find_band(scene_dir, band_id):
    for f in os.listdir(scene_dir):
        if f.lower().endswith(f"_b{band_id}.tif") or f.lower().endswith(f"_b{band_id}.tiff"):
            return os.path.join(scene_dir, f)
    raise FileNotFoundError(f"Band {band_id} not found in {scene_dir}")

# ===== LOAD PAN =====
pan_path = find_band(SCENE_DIR, PAN_BAND)
with rasterio.open(pan_path) as pan_ds:
    pan = pan_ds.read(1).astype(np.float32)
    pan_meta = pan_ds.meta.copy()

# ===== REPROJECT MS BANDS =====
ms_list = []
for b in MS_BANDS:
    ms_path = find_band(SCENE_DIR, b)
    with rasterio.open(ms_path) as ms_ds:
        ms_up = np.empty_like(pan, dtype=np.float32)
        reproject(
            source=rasterio.band(ms_ds, 1),
            destination=ms_up,
            src_transform=ms_ds.transform,
            src_crs=ms_ds.crs,
            dst_transform=pan_meta["transform"],
            dst_crs=pan_meta["crs"],
            resampling=Resampling.bilinear
        )
        ms_list.append(ms_up)

stack = np.stack(ms_list, axis=0)  # shape: (bands, H, W)
stack = stack / (np.nanpercentile(stack, 99.9, axis=(1,2), keepdims=True) + 1e-6)

# ===== BROVEY TRANSFORMATION =====
denominator = np.sum(stack, axis=0) + 1e-6  # avoid divide by zero
brovey_stack = stack * (pan / denominator)

# ===== SAVE OUTPUTS =====
for i, b in enumerate(MS_BANDS):
    out_path = os.path.join(OUT_DIR, f"LE09_2025_B{b:02d}_BROVEY.TIF")
    meta = pan_meta.copy()
    meta.update(count=1, dtype="float32", nodata=0)
    with rasterio.open(out_path, "w", **meta) as dst:
        dst.write(np.clip(brovey_stack[i], 0, np.nanmax(brovey_stack[i])).astype(np.float32), 1)
    print(f"✅ Band {b} saved → {out_path}")

print("\n🎯 Simple Brovey pansharpening complete.")
print(f"Outputs saved to: {OUT_DIR}")


In [ ]:
""" create composite of SR bands 1-7 (1999-2025) """

import os
import rasterio
import numpy as np

# === Input directory containing SR bands ===
SR_DIR = "SR_outputs/pansharpened_Brovey_basic"
OUT_PATH = os.path.join(SR_DIR, "LE09_SR_2025_COMPOSITE_B1toB7.tif")

bands = [1, 2, 3, 4, 5, 7]  # Landsat 7 SR bands

# Load first band for metadata
with rasterio.open(os.path.join(SR_DIR, f"LE09_2025_B01_BROVEY.TIF")) as src:
    meta = src.meta.copy()
    height, width = src.height, src.width

# Create empty composite array
stack = np.zeros((len(bands), height, width), dtype="float32")

# Load each band
for i, b in enumerate(bands):
    band_path = os.path.join(SR_DIR, f"LE09_2025_B0{b}_BROVEY.TIF")
    with rasterio.open(band_path) as src:
        stack[i] = src.read(1)

# Update metadata
meta.update({
    "count": len(bands),
    "dtype": "float32",
    "nodata": 0
})

# Save composite
with rasterio.open(OUT_PATH, "w", **meta) as dst:
    dst.write(stack)
    dst.descriptions = [f"B{b}" for b in bands]

print(f"✅ Composite saved to:\n  {OUT_PATH}")


In [ ]:
"""Compute MNDWI and create binary water mask with adaptive thresholding and morphological cleanup"""

import rasterio
import numpy as np
import os
from skimage.filters import threshold_otsu
from scipy.ndimage import binary_opening, binary_closing

INPUT_RASTER = ".tif"
OUT_MNDWI = INPUT_RASTER.replace(".tif", "_MNDWI.tif")
OUT_BIN = INPUT_RASTER.replace(".tif", "_MNDWI_BIN.tif")

# --- Load Green and SWIR1 ---
with rasterio.open(INPUT_RASTER) as src:
    green = src.read(2).astype("float32")  # Band 2 = Green
    swir1 = src.read(5).astype("float32")  # Band 5 = SWIR1
    meta = src.meta.copy()

# --- Compute MNDWI ---
mndwi = (green - swir1) / (green + swir1 + 1e-6)
mndwi = np.clip(mndwi, -1, 1)

# --- Adaptive Threshold (Otsu or Mean+Std) ---
valid = np.isfinite(mndwi)
try:
    t = threshold_otsu(mndwi[valid])
    print(f"🔹 Using Otsu threshold: {t:.3f}")
except Exception:
    t = np.nanmean(mndwi[valid]) + 0.5 * np.nanstd(mndwi[valid])
    print(f"🔹 Using adaptive mean+std threshold: {t:.3f}")

# --- Binary Mask ---
binary = np.zeros_like(mndwi, dtype="uint8")
binary[mndwi > t] = 1

# --- Morphological cleanup ---
binary = binary_closing(binary, structure=np.ones((3,3)))
binary = binary_opening(binary, structure=np.ones((3,3)))

# --- Save MNDWI and mask ---
meta.update(count=1, dtype="float32", nodata=np.nan)
with rasterio.open(OUT_MNDWI, "w", **meta) as dst:
    dst.write(mndwi, 1)

meta.update(dtype="uint8", nodata=0)
with rasterio.open(OUT_BIN, "w", **meta) as dst:
    dst.write(binary, 1)

print(f"✅ MNDWI and improved binary mask saved:\n  {OUT_MNDWI}\n  {OUT_BIN}")

In [ ]:
""" Prepare shoreline shapefile for DSAS analysis by adding required fields and saving as GeoJSON """

import geopandas as gpd
import os
import re

# === USER INPUT ===
input_shp = ".shp"   # your shapefile for shoreline
output_dir = "replace with output path"
os.makedirs(output_dir, exist_ok=True)

# === READ SHAPEFILE ===
gdf = gpd.read_file(input_shp)

# === EXTRACT YEAR FROM FILENAME ===
basename = os.path.basename(input_shp)
name, _ = os.path.splitext(basename)
m = re.search(r"(\d{4})", name)
if m:
    year = m.group(1)
    date_value = f"{year}-09-13"  # pick mid-year or your acquisition date
else:
    date_value = "1900-01-01"

# === ADD REQUIRED DSAS FIELDS ===
gdf["Date_"] = date_value
gdf["ID"] = range(1, len(gdf) + 1)
gdf["ShoreType"] = "Derived"
gdf["UNCERTAINTY"] = 7.5  # leave 0 if no uncertainty available
gdf["Group"] = "Wolin_Usedom"

# === SAVE AS GEOJSON ===
out_geojson = os.path.join(output_dir, name + ".geojson")
gdf.to_file(out_geojson, driver="GeoJSON")

print(f"✅ Converted: {input_shp}")
print(f"→ Saved DSAS-ready GeoJSON: {out_geojson}")


In [ ]:
"""Merge multiple shoreline GeoJSON files into a single GeoJSON for DSAS

"""

import geopandas as gpd
import pandas as pd
import os
import glob

# === USER INPUT ===
geojson_dir = ""
output_merged = os.path.join(geojson_dir, ".geojson")

# === FIND ALL GEOJSON FILES ===
geojson_files = glob.glob(os.path.join(geojson_dir, "*.geojson"))
if not geojson_files:
    raise FileNotFoundError("No GeoJSON files found in the folder.")

# === MERGE ===
gdf_list = [gpd.read_file(f) for f in geojson_files]
merged = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), crs=gdf_list[0].crs)

# === SAVE MERGED FILE ===
merged.to_file(output_merged, driver="GeoJSON")

print(f"✅ Merged {len(geojson_files)} GeoJSON files")
print(f"→ Saved combined shoreline dataset: {output_merged}")


In [ ]:
"""
Processing and compiling dataset into a single framework that is ready for inputting into the model_training_inference.ipynb script.

The user needs to check example_data.csv uploaded on github to know how the final dataset will look like and modify the model_training_inference.ipynb depending on the dataset

This script:
✅ Extracts NSM from DSAS GeoJSONs (sector-I, sector-II, sector-III)
✅ Merges with ERA5 wave height and direction data
✅ Merges with SLR data
✅ Adds coordinates and bearings for transects
✅ Outputs one clean CSV file ready for model input
"""

import os
import glob
import json
import pandas as pd
import geopandas as gpd
from pyproj import Transformer, Geod

# ======================================================
# USER INPUTS
# ======================================================

# Input DSAS folders for each sector
SECTOR_PATHS = {
    "sector-I": "",
    "sector-II": "",
    "sector-III": ""
}

# Output file paths
OUT_CSV = "Table_for_DL.csv"
FINAL_SORTED_CSV = "Table_for_DL_sorted.csv"
FINAL_COORDS_CSV = "Table_for_DL_with_coords.csv"

# ERA5 base data folder (wave height + direction)
ERA5_FOLDERS = {
    "sector-I": "usedom_ERA5_data",
    "sector-II": "Wolin_ERA5_data",
    "sector-III": "miedzywodzie_ERA5_data"
}

# Sea-level rise data
SLR_CSV = "slr_change_mm_intervals.csv"

# CRS transformation (adjust if your DSAS uses a different UTM zone)
SOURCE_CRS = "EPSG:32633"
TARGET_CRS = "EPSG:4326"

# ======================================================
# STEP 1 — Extract NSM from DSAS GeoJSONs (all sectors)
# ======================================================

all_records = []

for region_name, dsas_dir in SECTOR_PATHS.items():
    geojson_files = sorted(glob.glob(os.path.join(dsas_dir, "*.geojson")))
    if not geojson_files:
        print(f"⚠️ No GeoJSON files found for {region_name}")
        continue

    for geojson_file in geojson_files:
        try:
            gdf = gpd.read_file(geojson_file)

            transect_col = next((c for c in gdf.columns if "transect" in c.lower()), None)
            nsm_col = next((c for c in gdf.columns if "nsm" in c.lower()), None)
            if not transect_col or not nsm_col:
                print(f"⚠️ Skipping {geojson_file} — missing required columns.")
                continue

            # Extract start/end years from filename (e.g. 1987_1992_rates.geojson)
            base = os.path.basename(geojson_file)
            years = [int(s) for s in base.split("_") if s.isdigit()]
            if len(years) < 2:
                print(f"⚠️ Skipping {base}: could not extract start/end years.")
                continue

            start_year, end_year = years[:2]
            duration_yrs = end_year - start_year

            df = pd.DataFrame({
                "region": region_name,
                "transect_id": gdf[transect_col],
                "start_date": [f"{start_year}-06-01"] * len(gdf),
                "end_date": [f"{end_year}-06-01"] * len(gdf),
                "duration_yrs": [duration_yrs] * len(gdf),
                "nsm": gdf[nsm_col]
            })

            all_records.append(df)
            print(f"✅ Processed {base} ({len(df)} records) for {region_name}")

        except Exception as e:
            print(f"❌ Error reading {geojson_file}: {e}")

# Combine all extracted NSM data
final_df = pd.concat(all_records, ignore_index=True)
final_df.to_csv(OUT_CSV, index=False)
print(f"✅ NSM data combined for all sectors → {OUT_CSV}")

# ======================================================
# STEP 2 — Merge ERA5 wave height + direction data
# ======================================================

nsm_df = final_df.copy()
nsm_df["start_date"] = pd.to_datetime(nsm_df["start_date"])
nsm_df["end_date"] = pd.to_datetime(nsm_df["end_date"])
nsm_df["start_year"] = nsm_df["start_date"].dt.year
nsm_df["end_year"] = nsm_df["end_date"].dt.year

merged_all = []

for region, folder in ERA5_FOLDERS.items():
    wh_csv = os.path.join(folder, "wave_height_intervals.csv")
    wd_csv = os.path.join(folder, "wave_direction_intervals.csv")

    subset = nsm_df[nsm_df["region"] == region].copy()
    if not (os.path.exists(wh_csv) and os.path.exists(wd_csv)):
        print(f"⚠️ Skipping {region} (missing ERA5 files)")
        merged_all.append(subset)
        continue

    wh_df = pd.read_csv(wh_csv)
    wd_df = pd.read_csv(wd_csv)

    wh_df = wh_df.rename(columns={
        "mean": "swh_mean", "min": "swh_min", "max": "swh_max",
        "std": "swh_std", "p10": "swh_p10", "p50": "swh_p50", "p90": "swh_p90"
    })

    for df in [wh_df, wd_df]:
        df["start_year"] = df["start_year"].astype(int)
        df["end_year"] = df["end_year"].astype(int)

    merged = pd.merge(subset, wh_df, on=["start_year", "end_year"], how="left")
    overlapping = [c for c in wd_df.columns if c in merged.columns and c not in ["start_year", "end_year"]]
    wd_df = wd_df.drop(columns=overlapping)
    merged = pd.merge(merged, wd_df, on=["start_year", "end_year"], how="left")

    merged_all.append(merged)
    print(f"✅ ERA5 merged for {region} → {len(merged)} records")

wave_merged_df = pd.concat(merged_all, ignore_index=True)
wave_merged_df.to_csv(OUT_CSV, index=False)
print(f"✅ ERA5 data merged into main table → {OUT_CSV}")

# ======================================================
# STEP 3 — Merge Sea Level Rise (SLR) data
# ======================================================

# ======================================================
# STEP 3 — Extract Transect-Specific SLR from CMIP6 .nc files
# ======================================================

import xarray as xr
from scipy.spatial import cKDTree
import numpy as np
import os
import glob

def extract_slr_nc_files_station_based(nc_dir, transect_df):
    all_slr = []
    files = sorted(glob.glob(os.path.join(nc_dir, "*.nc")))

    for f in files:
        print(f"\nProcessing SLR file: {os.path.basename(f)}")
        ds = xr.open_dataset(f)
        var_name = list(ds.data_vars)[0]  # e.g., 'MSL'
        print(f"Variable in dataset: {var_name}")

        # Station coordinates
        station_x = ds["station_x_coordinate"].values
        station_y = ds["station_y_coordinate"].values
        coords = np.column_stack((station_x, station_y))
        tree = cKDTree(coords)
        print(f"Stations shape: {coords.shape}")

        # Transect coordinates
        tran_coords = np.column_stack((transect_df["lon"].values, transect_df["lat"].values))
        print(f"Transects shape: {tran_coords.shape}")

        # Query nearest station for each transect
        _, idxs = tree.query(tran_coords)
        print(f"Idxs min/max: {idxs.min()}/{idxs.max()}")

        # Extract SLR values for those stations safely
        slr_data = ds[var_name].values
        print(f"SLR data shape: {slr_data.shape}")

        if slr_data.ndim == 1:
            if idxs.max() >= slr_data.shape[0]:
                print("⚠️ idxs exceed slr_data.shape[0], clipping indices")
                idxs = np.clip(idxs, 0, slr_data.shape[0]-1)
            slr_vals = slr_data[idxs]
        elif slr_data.ndim == 2:
            # stations x time
            if slr_data.shape[1] == 1:
                if idxs.max() >= slr_data.shape[0]:
                    print("⚠️ idxs exceed slr_data.shape[0], clipping indices")
                    idxs = np.clip(idxs, 0, slr_data.shape[0]-1)
                slr_vals = slr_data[idxs, 0]
            else:
                if idxs.max() >= slr_data.shape[0]:
                    print("⚠️ idxs exceed slr_data.shape[0], clipping indices")
                    idxs = np.clip(idxs, 0, slr_data.shape[0]-1)
                # pick first time step as representative
                slr_vals = slr_data[idxs, 0]
        else:
            raise ValueError(f"Unexpected shape for {var_name}: {slr_data.shape}")

        # Convert to mm
        slr_vals_mm = slr_vals * 1000

        # Build dataframe for this year
        df_year = transect_df.copy()
        df_year["slr_mm"] = slr_vals_mm

        # Extract year from filename (assumes pattern: *_YYYY_MSL_v1.nc)
        fname = os.path.basename(f)
        year_str = fname.split("_")[-3]
        df_year["end_year"] = int(year_str)

        all_slr.append(df_year)

    return pd.concat(all_slr, ignore_index=True)

# Run extraction for historical and future SLR
slr_hist = extract_slr_nc_files_station_based(HIST_DIR, merged_coords_df)
slr_future = extract_slr_nc_files_station_based(FUT_DIR, merged_coords_df)
slr_df = pd.concat([slr_hist, slr_future], ignore_index=True)
slr_df.to_csv(FINAL_COORDS_CSV, index=False)
print(f"✅ SLR merged and final dataset saved → {FINAL_COORDS_CSV}")

# ======================================================
# STEP 4 — Sort by region and transect_id (custom order)
# ======================================================

region_order = ["sector-I", "sector-II", "sector-III"]
merged_df["region_order"] = merged_df["region"].apply(lambda x: region_order.index(x))
sorted_df = merged_df.sort_values(by=["region_order", "transect_id"]).drop(columns="region_order")
sorted_df.to_csv(FINAL_SORTED_CSV, index=False)
print(f"✅ Sorted data saved → {FINAL_SORTED_CSV}")

# ======================================================
# STEP 5 — Add coordinates + bearing (user’s original function)
# ======================================================

transformer = Transformer.from_crs(SOURCE_CRS, TARGET_CRS, always_xy=True)
geod = Geod(ellps="WGS84")

def extract_coords_and_bearing(geojson_path, region_name):
    with open(geojson_path, "r") as f:
        data = json.load(f)
    
    records = []
    for feature in data["features"]:
        props = feature["properties"]
        tid = props.get("TransectId") or props.get("transect_id")
        geom = feature["geometry"]

        if geom is None or "coordinates" not in geom:
            continue
        coords = geom["coordinates"]
        if not coords or len(coords) < 2:
            continue

        x1, y1 = coords[0]
        x2, y2 = coords[-1]
        lon1, lat1 = transformer.transform(x1, y1)
        lon2, lat2 = transformer.transform(x2, y2)

        az12, az21, dist = geod.inv(lon1, lat1, lon2, lat2)
        bearing_deg = az12 % 360
        lon_mid, lat_mid = (lon1 + lon2) / 2, (lat1 + lat2) / 2

        records.append({
            "region": region_name,
            "transect_id": tid,
            "lon": lon_mid,
            "lat": lat_mid,
            "bearing_deg": bearing_deg
        })
    
    return pd.DataFrame(records)

coord_all = []
for region, path in SECTOR_PATHS.items():
    geojsons = glob.glob(os.path.join(path, "*rates*.geojson"))
    if not geojsons:
        print(f"⚠️ No GeoJSON for {region}")
        continue
    geojson_path = geojsons[0]
    print(f"📂 Extracting coordinates & bearings from {geojson_path}")
    region_coords = extract_coords_and_bearing(geojson_path, region)
    coord_all.append(region_coords)

coords_df = pd.concat(coord_all, ignore_index=True)
merged_coords_df = sorted_df.merge(coords_df, on=["region", "transect_id"], how="left")
merged_coords_df.to_csv(FINAL_COORDS_CSV, index=False)
print(f"✅ Final dataset with coordinates saved → {FINAL_COORDS_CSV}")

print("\n🎯 All preprocessing complete — dataset ready for model training.")
